In [1]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)
import numpy as np
import torch

import os
os.getcwd()

'/home/dan/NeurIPS2023-One-Fits-All1/dataset/Dataset-2'

In [2]:
# Load and process the datasets
calendar = pd.read_csv("./calendar.csv")
sell_prices = pd.read_csv("./sell_prices.csv")
sales_data = pd.read_csv("./sales_train_evaluation.csv")

# Keep the original columns intact for indexing later
calendar['date_id'] = range(1, len(calendar) + 1)

# Reshape sales data to long format while retaining the original columns
sales_long = pd.melt(
    sales_data,
    id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='units'
)
sales_long['d'] = sales_long['d'].str.replace('d_', '').astype(int)

# Merge sales data with calendar data
merged_data = sales_long.merge(calendar, left_on='d', right_on='date_id', how='left')

# Merge with sell prices data
final_data = merged_data.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

# Drop unnecessary columns, but retain key indexing columns
final_data = final_data.drop(columns=['d', 'wm_yr_wk'])

In [3]:
tensor = pd.read_csv('./prepared_sale_dataset.csv')

In [4]:
tensor

,units,month,year,sell_price,dept_id_FOODS_1,dept_id_FOODS_2,dept_id_FOODS_3,dept_id_HOBBIES_1,dept_id_HOBBIES_2,dept_id_HOUSEHOLD_1,dept_id_HOUSEHOLD_2,cat_id_FOODS,cat_id_HOBBIES,cat_id_HOUSEHOLD,state_id_CA,state_id_TX,state_id_WI,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,event_name_1_Chanukah End,event_name_1_Christmas,event_name_1_Cinco De Mayo,event_name_1_ColumbusDay,event_name_1_Easter,event_name_1_Eid al-Fitr,event_name_1_EidAlAdha,event_name_1_Father's day,event_name_1_Halloween,event_name_1_IndependenceDay,event_name_1_LaborDay,event_name_1_LentStart,event_name_1_LentWeek2,event_name_1_MartinLutherKingDay,event_name_1_MemorialDay,event_name_1_Mother's day,event_name_1_NBAFinalsEnd,event_name_1_NBAFinalsStart,event_name_1_NewYear,event_name_1_OrthodoxChristmas,event_name_1_OrthodoxEaster,event_name_1_Pesach End,event_name_1_PresidentsDay,event_name_1_Purim End,event_name_1_Ramadan starts,event_name_1_StPatricksDay,event_name_1_SuperBowl,event_name_1_Thanksgiving,event_name_1_ValentinesDay,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_type_2_Cultural,event_type_2_Religious,snap_CA_1,snap_TX_1,snap_WI_1
0,3,1,2011,2.00,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0,1,2011,2.00,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,0,1,2011,2.00,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1,2,2011,2.00,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False
4,4,2,2011,2.00,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,0,5,2016,5.94,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
59181086,0

In [4]:
# Move 'sell_price' and 'units' to the end of the DataFrame
cols = [col for col in tensor.columns if col not in ['sell_price', 'units']] + ['sell_price', 'units']
tensor = tensor[cols]

In [5]:
# Get the number of unique values for each index level
item_ids = final_data['item_id'].nunique()
store_ids =final_data['store_id'].nunique() 
date_ids = final_data['date_id'].nunique()
features = tensor.shape[1]  # Number of columns/features
print('item_ids',item_ids,"store_ids",store_ids,"date_ids",date_ids,"features",features)
# Convert the DataFrame to a numpy array and then reshape to 4D
data_array =tensor.to_numpy(dtype=np.float32).reshape(item_ids, store_ids, date_ids, features)

# Convert the numpy array to a PyTorch tensor
data_tensor_4d = torch.tensor(data_array)

# Check the shape of the tensor
print("4D Tensor Shape:", data_tensor_4d.shape)  # Should be (item_ids, store_ids, date_ids, features)

item_ids 3049 store_ids 10 date_ids 1941 features 67
4D Tensor Shape: torch.Size([3049, 10, 1941, 67])


In [6]:
reshaped_tensor = data_tensor_4d.permute(0, 1, 3, 2)

In [7]:
torch.save(reshaped_tensor,'./sale_tensor.pt')